In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import utils

__author__ = "Olivares Castillo José Luis"

print("TensorFlow",tf.VERSION)
if tf.test.gpu_device_name():
  print("GPU disponible")

tf.logging.set_verbosity(tf.logging.INFO)
tf.enable_eager_execution()

/home/olivares/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


TensorFlow 1.7.0


In [2]:
source_file = open("datasets/en-it/en.crawl.embeddings.300d.5k.train")
src_words,src_matrix = utils.read(source_file,is_zipped=False)

In [3]:
target_file = open("datasets/en-it/it.embeddings.300d.5k.train")
trg_words,trg_matrix = utils.read(target_file,is_zipped=False)

In [4]:
trg_matrix.shape,src_matrix.shape,src_matrix.dtype.type

((5000, 300), (5000, 300), numpy.float32)

In [5]:
def next_batch(x,y, step, batch_size):
    return x[batch_size * step:batch_size * step + batch_size],y[batch_size * step:batch_size * step + batch_size]

https://ikhlestov.github.io/posts/rbm-based-autoencoders-with-tensorflow/
https://gist.github.com/blackecho/db85fab069bd2d6fb3e7

In [12]:
tf.reset_default_graph()
tf.set_random_seed(42)
tf.logging.set_verbosity(tf.logging.INFO)

LEARNING_RATE = 1
EPOCHS = 15000
# Dimensión de vectores de entrada (número de neuronas en capa de entrada).
NODES_INPUT = src_matrix.shape[1]

# Número de neuronas en capas ocultas.
NODES_H1 = 270
NODES_H2 = 360

# Dimensión de vectores de salida (número de neuronas en capa de salida).
NODES_OUTPUT = trg_matrix.shape[1]

#https://www.tensorflow.org/programmers_guide/eager#build_a_model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(units=NODES_H1,
                         input_shape=(NODES_INPUT,),
                         activation=tf.nn.elu,
                         kernel_initializer=tf.contrib.layers.xavier_initializer(),
                         use_bias=True,
                         bias_initializer=tf.keras.initializers.Zeros()),
    tf.keras.layers.Dense(units=NODES_H1,
                         activation=tf.nn.tanh,
                         kernel_initializer=tf.contrib.layers.xavier_initializer(),
                         use_bias=True,
                         bias_initializer=tf.keras.initializers.Zeros()),
    
])

h1 = tf.keras.layers.Dense(
    units=NODES_H1,
    activation=tf.nn.elu,
    kernel_initializer=tf.contrib.layers.xavier_initializer(),
    use_bias=True,
    bias_initializer=tf.zeros_initializer()
)

output=tf.keras.layers.Dense(
    units=NODES_OUTPUT,
    activation=tf.nn.tanh,
    kernel_initializer=tf.contrib.layers.xavier_initializer(),
    use_bias=True,
    bias_initializer=tf.zeros_initializer()
)

# Entrenamiento

In [ ]:
%%time
tf.reset_default_graph()
tf.set_random_seed(42)
print("TensorFlow v{}".format(tf.__version__))
print(tf.test.gpu_device_name())

LEARNING_RATE = 1
EPOCHS = 15000
# Dimensión de vectores de entrada (número de neuronas en capa de entrada).
NODES_INPUT = source_vectores.shape[1]

# Número de neuronas en capas ocultas.
NODES_H1 = 270   #70 - 20 - 15
NODES_H2 = 360  # 42 - 20
NODES_H3 = 360  # 42 - 20
NODES_H4 = 110  # 42 - 20
NODES_H5 = 950

# Dimensión de vectores de salida (número de neuronas en capa de salida).
NODES_OUTPUT = target_vectores.shape[1]

DROP = 1

model = "model2250"


with tf.name_scope('input'):
    X = tf.placeholder(shape=[None, NODES_INPUT],dtype=tf.float64, name='input_es')
    y = tf.placeholder(shape=[None, NODES_OUTPUT],dtype=tf.float64, name='target_na')


kprob = tf.placeholder(tf.float64,name='dropout_prob')


def activation_function(layer, act, name, alpha=tf.constant(0.001, dtype=tf.float64)):
    if act.__eq__("leaky_relu"):
        return tf.nn.leaky_relu(layer, alpha, name=name)
    elif act.__eq__("softmax"):
        return tf.nn.softmax(layer, name=name)
    elif act.__eq__("sigmoid"):
        return tf.nn.sigmoid(layer, name=name)
    elif act.__eq__("tanh"):
        return tf.nn.tanh(layer, name=name)
    elif act.__eq__("elu"):
        return tf.nn.elu(layer,name=name)
    elif act.__eq__("selu"):
        return tf.nn.selu(layer,name=name)
    return tf.nn.relu(layer, name=name)


# Se definen las capas.
w1 = tf.get_variable(name="W1", dtype=tf.float64,
                                shape=[NODES_INPUT, NODES_H1],
                                initializer=tf.truncated_normal_initializer(stddev=0.1,seed=41),
                                #initializer=W1_,
                                use_resource=True,
                                regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
                    )
b1 = tf.Variable(tf.zeros(shape=[NODES_H1], dtype=tf.float64), name="b1")
fc1 = activation_function(tf.nn.xw_plus_b(X,w1,b1), "tanh", "fc1")
fc1 = tf.nn.dropout(fc1,kprob,seed=42)


w2 = tf.get_variable(name="W2", dtype=tf.float64,
                                shape=[NODES_H1, NODES_OUTPUT],
                                initializer=tf.truncated_normal_initializer(stddev=0.1,seed=42),
                                #initializer=W3_,
                                use_resource=True,
                                regularizer = tf.contrib.layers.l2_regularizer(scale=0.1)
                    )
b2 = tf.Variable(tf.zeros(shape=[NODES_OUTPUT], dtype=tf.float64), name="b2")

nah_predicted = activation_function(tf.nn.xw_plus_b(fc1,w2,b2),"tanh","nah_predicted")



# Función de error
loss = tf.reduce_mean(tf.squared_difference(nah_predicted, y), name="loss")


tf.summary.scalar("loss", loss)


# backprop
optimiser = tf.train.AdagradOptimizer(learning_rate=LEARNING_RATE)


# Compute gradients
gradients, variables = zip(*optimiser.compute_gradients(loss))

gradients, _ = tf.clip_by_global_norm(gradients, 5.0)

# Apply processed gradients to optimizer.
train_op = optimiser.apply_gradients(zip(gradients, variables))


# Accuracy 
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        # Se compara salida de la red neuronal con el vector objetivo.
        correct_prediction = tf.equal(tf.argmax(nah_predicted, 1), tf.argmax(y, 1))
    with tf.name_scope('accuracy'):
        # Se calcula la precisión.
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))
    tf.summary.scalar('accuracy', accuracy)


LOGPATH = "logs/model"
print("logpath:", LOGPATH)


# Se crea la sesión
config = tf.ConfigProto(log_device_placement=True)
sess =  tf.Session(config=config)

# Se ponen los histogramas y valores de las gráficas en una sola variable.
summaryMerged = tf.summary.merge_all()

# Escribir a disco el grafo generado y las gráficas para visualizar en TensorBoard.
writer = tf.summary.FileWriter(LOGPATH, sess.graph)

# Se inicializan los valores de los tensores.
init = tf.global_variables_initializer()

# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Ejecutando sesión
sess.run(init)


for j in range(EPOCHS):
    
    for i in range(0,5):
      source_batch, target_batch = next_batch(source_vectores,target_vectores, i, 1000)
      _loss, _, sumOut = sess.run([loss, train_op, summaryMerged],
                                feed_dict={X: source_batch, y: target_batch,kprob:DROP})
      

    writer.add_summary(sumOut, j)
    
    if (j % 500) == 0:
      #train_accuracy = accuracy.eval(session=sess, feed_dict={X: tsource_vectores, y: ttarget_vectores,kprob:1})
      print("Epoch:", j, "/", EPOCHS, "\tLoss:",_loss)#,"\tAccuracy:", train_accuracy)
        
        
        

SAVE_PATH = "./"+model+".ckpt"
print("save path",SAVE_PATH)
save_model = saver.save(sess, SAVE_PATH)
print("Model saved in file: %s", SAVE_PATH)
writer.close()

# Descargar modelo generado

In [ ]:
files.download("/content/checkpoint")

In [ ]:
files.download("/content/"+model+".ckpt.meta")

In [ ]:
files.download("/content/"+model+".ckpt.index")

In [ ]:
files.download("/content/"+model+".ckpt.data-00000-of-00001")

In [ ]:
! rm -rf logs/model